# Convert fore names to simplified (one word) fore names

In [1]:
import collections
import string
import pandas

import pubmedpy.names

In [2]:
name_df_paths = [
    'data/pubmed/authors.tsv.xz',
    'data/pmc/authors.tsv.xz',
    'data/iscb/keynotes.tsv',
]
path_to_df = {}
for path in name_df_paths:
    name_df = pandas.read_csv(path, keep_default_na=False, na_values=[''], sep='\t')
    path_to_df[path] = name_df

In [3]:
def simplify_names(dfs, name_column):
    names = pandas.concat(df[name_column] for df in dfs)
    simple_name_df = names.value_counts(dropna=True).reset_index()
    simple_name_df.columns = [name_column, 'n_authors']
    assert simple_name_df[name_column].isna().sum() == 0
    simplifier = getattr(pubmedpy.names, f'simplify_{name_column}')
    simple_name_df[f'{name_column}_simple'] = simple_name_df[name_column].map(lambda x: simplifier(x, lower=True))
    simple_name_df = simple_name_df.sort_values(name_column)
    return simple_name_df

In [4]:
simple_fore_name_df = simplify_names(path_to_df.values(), 'fore_name')
simple_fore_name_df

,fore_name,n_authors,fore_name_simple
28784,(Max) Zong-Ming,1,max
35774,", Yuxin",1,yuxin
25,A,357,None
2520,A A,15,None
16721,A Aldo,2,aldo
...,...,...,...
23135,Špela,2,špela
15905,Žiga,2,žiga
30813,Živa,1,živa
14976,Živadin,2,živadin


In [5]:
simple_last_name_df = simplify_names(path_to_df.values(), 'last_name')
simple_last_name_df

,last_name,n_authors,last_name_simple
34480,Michor,1,michor
35045,Rozowsky,1,rozowsky
38736,Slater,1,slater
39201,'t Hart,1,t hart
2403,'t Hoen,14,t hoen
...,...,...,...
32596,Žárský,2,žárský
38141,železný,1,železný
38363,žurauskienė,1,žurauskienė
27069,‘t Hoen,2,‘t hoen


In [6]:
simple_fore_name_df.sample(n=20, random_state=0).sort_values('n_authors', ascending=False)

,fore_name,n_authors,fore_name_simple
201,Qiang,116,qiang
238,Pascal,103,pascal
387,Jean-Philippe,74,jean-philippe
1018,Pei,33,pei
1679,Lynette,22,lynette
2081,Bastian,18,bastian
2280,Harold R,17,harold
4028,Chih-Hung,10,chih-hung
6807,Mohammad Ali,6,mohammad
6900,Dmitry A.,6,dmitry


In [7]:
simple_last_name_df.sample(n=20, random_state=0).sort_values('n_authors', ascending=False)

,last_name,n_authors,last_name_simple
2744,Sundmacher,13,sundmacher
4614,Nassar,9,nassar
4629,Rogozin,8,rogozin
9007,Zafrir,5,zafrir
8644,Lisewski,5,lisewski
14017,Bloch,3,bloch
14729,Van Belle,3,van belle
22944,Mudvari,2,mudvari
16684,Guilliams,2,guilliams
19493,Buer,2,buer


In [8]:
# Most common last names
(
    simple_last_name_df
    .groupby('last_name_simple')
    .n_authors.sum()
    .sort_values(ascending=False)
    .reset_index()
    .head(3)
)

,last_name_simple,n_authors
0,wang,3184
1,li,2670
2,zhang,2620


In [9]:
simple_fore_name_df.to_csv('data/names/fore-names.tsv.xz', sep='\t', index=False)
simple_last_name_df.to_csv('data/names/last-names.tsv.xz', sep='\t', index=False)